In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
sqlitedb_path = '../data/raw/raw_data_project_m1.db'

In [3]:
engine = create_engine(f'sqlite:///{sqlitedb_path}')

In [4]:
personal_info_raw = pd.read_sql_query("SELECT * FROM personal_info", engine)
personal_info_raw.head()

,uuid,age,gender,dem_has_children,age_group
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,male,NO,40_65
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,male,yES,40_65
2,83127080-da3d-0133-c74f-0a81e8b09a82,32 years old,male,nO,26_39
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45 years old,Male,YES,40_65
4,24954a70-db98-0133-4a64-0a81e8b09a82,41 years old,Fem,yES,40_65


In [5]:
country_info_raw = pd.read_sql_query("SELECT * FROM country_info", engine)
country_info_raw.head()

,uuid,country_code,rural
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,countryside
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,city
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,Country
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,city


In [6]:
career_info_raw = pd.read_sql_query("SELECT * FROM career_info", engine)
career_info_raw.head()

,uuid,dem_education_level,dem_full_time_job,normalized_job_code
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,no,no,None
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,861a9b9151e11362eb3c77ca914172d0
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,no,None
3,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c
4,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f


In [7]:
poll_info_raw = pd.read_sql_query("SELECT * FROM poll_info", engine)
poll_info_raw.head()

,uuid,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,I know something about it,I would not vote,None of the above,None of the above,None of the above
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working
2,83127080-da3d-0133-c74f-0a81e8b09a82,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...
3,15626d40-db13-0133-ea5c-0a81e8b09a82,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above
4,24954a70-db98-0133-4a64-0a81e8b09a82,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...


In [8]:
#  ---------- MERGED ----------

merge_1 = pd.merge(personal_info_raw, country_info_raw, how='inner', on='uuid')
merge_2 = pd.merge(merge_1, career_info_raw, how='inner', on='uuid')
tables_merged_raw = pd.merge(merge_2, poll_info_raw, how='inner', on='uuid') # ---> Dataframe merged

In [9]:
tables_merged_raw.rename(columns={'age': 'Age (years)'}, inplace=True)

In [10]:
#  ---------- Clean AGE column ----------

for i in range(len(tables_merged_raw)):
    if 'years' in tables_merged_raw.loc[i, 'Age (years)']:
        tables_merged_raw.loc[i, 'Age (years)'] = int(tables_merged_raw.loc[i, 'Age (years)'].replace(' years old', ''))
    else:
        tables_merged_raw.loc[i, 'Age (years)'] = 2016 - int(tables_merged_raw.loc[i, 'Age (years)'])
        

In [11]:
#tables_merged_raw

In [12]:
jobs_list = set(tables_merged_raw['normalized_job_code'].to_list())

In [13]:
jobs_list = list(tables_merged_raw['normalized_job_code'].unique())
jobs_list[:10] # -----> Esto habra que cruzarlo con el df de la API con left join

[None,
 '861a9b9151e11362eb3c77ca914172d0',
 '049a3f3a2b5f85cb2971ba77ad66e10c',
 'f4b2fb1aa40f661488e2782b6d57ad2f',
 '27af8700f5577cec835acee2cb90a2ff',
 'c1b670eba9ccb65e7c99f7da116d5b9c',
 'd71c4164c99a5f9bca773e755a72f40f',
 '2ee80cf30e2483dfe0947adc0428f8c5',
 '897ef18a9321616495831a4b4b0c1bb2',
 'a003702c0cea282e17967772dd83de5e']

In [14]:
#  ---------- API ----------

In [15]:
import requests

#http://api.dataatwork.org/v1/jobs?offset=46296&limit=1

# ----- Get the number of jobs -----

response = requests.get('http://api.dataatwork.org/v1/jobs?offset=0&limit=1')

json_data = response.json()

number_of_jobs = int(json_data[1]['links'][3]['href'].replace('/jobs?offset=','').replace('&limit=1',''))
number_of_requests = number_of_jobs // 500
last_request = number_of_jobs % 500

In [16]:
# ----- Get jsons ----- REQUEST


response_500s = [requests.get(f'http://api.dataatwork.org/v1/jobs?offset={500 * (request-1)}&limit=500') for request in range(number_of_requests)]
response_last = requests.get(f'http://api.dataatwork.org/v1/jobs?offset={number_of_jobs - last_request}&limit=500')

In [17]:
# ----- Get jsons ----- JSONS

json_data_500s = [response.json()[:-1] for response in response_500s]
json_data_last = response_last.json()[:-1]

# ----- Flatten and join -----
import itertools

json_data_500s_flat = list(itertools.chain(*json_data_500s))
json_data = json_data_500s_flat + json_data_last # ---->  List of all jobs 

jobs_df = pd.DataFrame(json_data)
jobs_df = jobs_df.rename(columns={'uuid': 'job_uuid'})
jobs_df.head()

,normalized_job_title,parent_uuid,title,job_uuid
0,1st grade teacher,51cc95467c1844c7dc1d923265c023b2,1st Grade Teacher,26bc4486dfd0f60b3bb0d8d64e001800
1,1st pressman,96f1cf15fe5880d8660c70f0756944ab,1st Pressman,9be6e14491305132c175cb1781a3356c
2,1st pressman on web press,96f1cf15fe5880d8660c70f0756944ab,1st Pressman On Web Press,d38d16bab3f9f25ae34ae3e2aad254d4
3,21 dealer,4be080220c824dcb435d59ebf1c3ce80,21 Dealer,8576a78b5c1cae3a567ca7fd34500357
4,2nd grade teacher,51cc95467c1844c7dc1d923265c023b2,2nd Grade Teacher,a59278d7f653db37e54a270345a91a05


In [18]:
#  ---------- WEB SCRAPING ----------

In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [20]:
url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'

In [21]:
html = requests.get(url).content
soup = BeautifulSoup(html, 'lxml')

In [22]:
tables = soup.find_all('table')

In [23]:
countries_raw = [table.find_all('td') for table in tables]

countries_flatted = list(itertools.chain(*countries_raw))


countries = [country.text.replace('\n','') for country in countries_flatted]
countries_clean = [country for country in countries if country != '' and country != ' \xa0']
countries_clean = [re.sub('\[[0-9]\]','',country).replace('*','') for country in countries_clean]


countries_names = [country.strip() for country in countries_clean[::2]]
countries_codes = [country.replace('(','').replace(')','').strip() for country in countries_clean [1::2]]
countries_dict = {countries_codes[i]: countries_names[i] for i in range(len(countries_names))}
countries_dict['GB'] = countries_dict['UK']
countries_dict['GR'] = countries_dict['EL']

In [24]:
# countries_dict
# jobs_df
# tables_merged_raw

In [25]:
#  ---------- TODAS LAS TABLAS JUNTAS ----------

tables_merged_jobs_raw = pd.merge(tables_merged_raw, jobs_df, how='left', left_on='normalized_job_code' ,right_on='job_uuid')
tables_merged_jobs_raw.head()
tables_merged_jobs_raw['Country'] = [countries_dict[f'{country}'] for country in tables_merged_jobs_raw['country_code']]

In [26]:
tables_merged_jobs_raw.head()

,uuid,Age (years),gender,dem_has_children,age_group,country_code,rural,dem_education_level,dem_full_time_job,normalized_job_code,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,normalized_job_title,parent_uuid,title,job_uuid,Country
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61,male,NO,40_65,AT,countryside,no,no,None,I know something about it,I would not vote,None of the above,None of the above,None of the above,NaN,NaN,NaN,NaN,Austria
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57,male,yES,40_65,AT,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,automatic data processing planner,0148f61d4227497728ce33490843d056,Automatic Data Processing Planner,861a9b9151e11362eb3c77ca914172d0,Austria
2,83127080-da3d-0133-c74f-0a81e8b09a82,32,male,nO,26_39,AT,city,None,no,None,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...,NaN,NaN,NaN,NaN,Austria
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45,Male,YES,40_65,AT,Country,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above,data coordinator,0b9dd32a367f4562ec77b993053d1910,Data Coordinator,049a3f3a2b5f85cb2971ba77ad66e10c,Austria
4,24954a70-db98-0133-4a64-0a81e8b09a82,41,Fem,yES,40_65,AT,city,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...,database developer,b90ca4df5690002377a7b0f1f3d40781,Database Developer,f4b2fb1aa40f661488e2782b6d57ad2f,Austria


In [27]:
#  ---------- FILTRAR  nos quedamos con las filas que tienen trabajo ----------

filter_jobs = ~tables_merged_jobs_raw['title'].isnull()
table_merged_raw = tables_merged_jobs_raw[filter_jobs]

table_merged_raw.head()

,uuid,Age (years),gender,dem_has_children,age_group,country_code,rural,dem_education_level,dem_full_time_job,normalized_job_code,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,normalized_job_title,parent_uuid,title,job_uuid,Country
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57,male,yES,40_65,AT,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,automatic data processing planner,0148f61d4227497728ce33490843d056,Automatic Data Processing Planner,861a9b9151e11362eb3c77ca914172d0,Austria
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45,Male,YES,40_65,AT,Country,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above,data coordinator,0b9dd32a367f4562ec77b993053d1910,Data Coordinator,049a3f3a2b5f85cb2971ba77ad66e10c,Austria
4,24954a70-db98-0133-4a64-0a81e8b09a82,41,Fem,yES,40_65,AT,city,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...,database developer,b90ca4df5690002377a7b0f1f3d40781,Database Developer,f4b2fb1aa40f661488e2782b6d57ad2f,Austria
5,28583060-dbe2-0133-ff9c-0a81e8b09a82,26,FeMale,NO,26_39,AT,countryside,high,yes,27af8700f5577cec835acee2cb90a2ff,I understand it fully,I would vote against it,A basic income would not affect my work choices,It encourages financial independence and self-...,It is impossible to finance | It might encoura...,data entry specialist,bf88358c56bb6cbe7eabec38ae333d19,Data Entry Specialist,27af8700f5577cec835acee2cb90a2ff,Austria
7,dd743bb0-d9a5-0133-53b2-0a81e8b09a82,47,Male,NO,40_65,AT,urban,medium,yes,c1b670eba9ccb65e7c99f7da116d5b9c,I know something about it,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working | Fo...,database architect,596226d5f9ef63de41a852826d483d19,Database Architect,c1b670eba9ccb65e7c99f7da116d5b9c,Austria


In [28]:
table = table_merged_raw[['Country', 'title', 'Age (years)']]
table_df = pd.DataFrame(table.groupby(['Country', 'title', 'Age (years)']).size()).reset_index().set_index('Country')

table_df.rename(columns={'title': 'Job Title', 0: 'Quantity'}, inplace=True)

table_df['Percentage'] = 100 * table_df['Quantity']/len(table_df)

table_df.head()

,Job Title,Age (years),Quantity,Percentage
Country,,,,
Austria,Automatic Data Processing Customer Liaison (AD...,31,1,0.018882
Austria,Automatic Data Processing Customer Liaison (AD...,40,1,0.018882
Austria,Automatic Data Processing Planner,48,1,0.018882
Austria,Automatic Data Processing Planner,57,1,0.018882
Austria,Automatic Data Processing Systems Security Spe...,27,1,0.018882
